# Self join: intuition

Comme son nom l'indique, un self join revient à joindre une table avec elle même !

Ce n'est donc pas vraiment un type de join comme les cross, inner, left ou full outer.

En effet, vous pouvez tout à fait utiliser n'importe lequel de ces types de joins pour faire votre self-join.

<blockquote> Mais pourquoi voudrait-on joindre une table avec elle-même ?! </blockquote>

Il y a plusieurs cas de figure, mais le plus répandu est celui des structures hiérarchiques. Imaginez une table qui contient l'id de vos employés (oui, dans quelques années vous en aurez :p), et l'id de leurs managers:

<img src="images/0intuition_self_join.png" />

<i>(Oui, je sais, un id c'est sensé être une clé unique mais on commence simple ;) )</i>

Vous aimeriez avoir le n+2 de chaque employé en face de celui-ci.

Comment faire... ?


Il "suffit" de lister toutes les combinaisons possibles:

<img src="images/self_join_intuition1.png" />

---

Puis de regarder dans quel cas il y a un match entre l'id du superviseur et l'employee_id:

<img src="images/self_join_intuition2.png" />

Ainsi, on peut voir que le n+2 de Toufik c'est Jean-Nicolas ;) 

Sylvie, qui a pour manager Jean-Nicolas, n'a pas de N+2. En effet Jean-Nicolas n'a pas de manager, c'est le CEO de la boîte !

# Self join: exemple introductif en code

In [5]:
import pandas as pd

In [6]:
# Table des employés
employees = {
    'employee_id': [11, 12, 13, 14, 15],
    'employee_name': ["Sophie", "Sylvie", "Daniel", "Kaouter", "David"],
    'manager_id': [13, None, 12, 13, 11],
}

df_employees = pd.DataFrame(employees)
df_employees

,employee_id,employee_name,manager_id
0,11,Sophie,13.0
1,12,Sylvie,NaN
2,13,Daniel,12.0
3,14,Kaouter,13.0
4,15,David,11.0


In [7]:
employee_with_manager = (
    df_employees
    .merge(df_employees,
           left_on="manager_id",
           right_on="employee_id",
           suffixes=["_emp", "_man"],
           how="left"
          )
    # .drop("employee_id_man", axis=1)
)

employee_with_manager

,employee_id_emp,employee_name_emp,manager_id_emp,employee_id_man,employee_name_man,manager_id_man
0,11,Sophie,13.0,13.0,Daniel,12.0
1,12,Sylvie,NaN,NaN,NaN,NaN
2,13,Daniel,12.0,12.0,Sylvie,NaN
3,14,Kaouter,13.0,13.0,Daniel,12.0
4,15,David,11.0,11.0,Sophie,13.0


In [8]:
# Autant de fois qu'on veut ;)

(
    employee_with_manager
    .merge(df_employees,
           left_on="manager_id_man",
           right_on="employee_id",
           suffixes=["_n+1", "_n+2"],
           how="left"
          )
    # .drop("employee_id_man", axis=1)
)

,employee_id_emp,employee_name_emp,manager_id_emp,employee_id_man,employee_name_man,manager_id_man,employee_id,employee_name,manager_id
0,11,Sophie,13.0,13.0,Daniel,12.0,12.0,Sylvie,NaN
1,12,Sylvie,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,Daniel,12.0,12.0,Sylvie,NaN,NaN,NaN,NaN
3,14,Kaouter,13.0,13.0,Daniel,12.0,12.0,Sylvie,NaN
4,15,David,11.0,11.0,Sophie,13.0,13.0,Daniel,12.0


<img src="images/employee_superviseurs.png" />

# Exercice: faire la même chose en SQL

In [9]:
import duckdb

In [12]:
%load solutions/1self_join_employees.py

┌─────────────┬───────────────┬────────────┬─────────────┬───────────────┬────────────┐
│ employee_id │ employee_name │ manager_id │ employee_id │ employee_name │ manager_id │
│    int64    │    varchar    │   double   │    int64    │    varchar    │   double   │
├─────────────┼───────────────┼────────────┼─────────────┼───────────────┼────────────┤
│          11 │ Sophie        │       13.0 │          13 │ Daniel        │       12.0 │
│          13 │ Daniel        │       12.0 │          12 │ Sylvie        │       NULL │
│          14 │ Kaouter       │       13.0 │          13 │ Daniel        │       12.0 │
│          15 │ David         │       11.0 │          11 │ Sophie        │       13.0 │
│          12 │ Sylvie        │       NULL │        NULL │ NULL          │       NULL │
└─────────────┴───────────────┴────────────┴─────────────┴───────────────┴────────────┘